In [ ]:
import os
import textract
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Préparation du corpus

### Dézipper nos fichiers dans un répertoire dédié

In [ ]:
!mkdir data
!unzip camille_962_congo.zip 
!mv *.txt data
!unzip camille_50_congo.zip
!mv *.txt data

#### Vérification du nombre de fichiers (il en faut 1012)

In [ ]:
#vérifier le nombre de fichier
path = "data/"
files = sorted(os.listdir(path))
len(files)

#### Création d'un fichier unique reprenant nos 1012 fichiers

In [ ]:
!rm data/_README.txt
!cat data/*.txt > data/all.txt

# Première exploration : la distribution de nos données par journaux et par année

### Distribution du corpus par journaux :

In [ ]:
path = "data"
files = sorted(os.listdir(path))
len(files)

In [ ]:
chaine = 'KB_JB838_1887-12-22_01-00001'
type(chaine)
# la méthode split
chaine_split = chaine.split('_')
chaine_split
# Accéder à la date
date = chaine_split[2]
# Accéder à l'année
date_split = date.split('-')
year = date_split[0]
year[:3] + '0s'
all_years = [str(year) for year in range(1885, 1914)]

In [ ]:
count_decade = defaultdict(int)
count_month = defaultdict(int)
count_newspapers = defaultdict(int)
count_years = defaultdict(int)
covered_years = set()

for f in files:
    if "_" in f and f.endswith("txt"):
        elems = f.split("_")
        
        newspaper = elems[1]
        
        year = elems[2].split("-")[0]
        covered_years.add(year)
        decade = year[:3] + "0s"
        
        month = int(elems[2].split("-")[1])
        
        count_decade[decade] += 1
        count_newspapers[newspaper] += 1
        count_month[month] += 1
        count_years[year] += 1
    else:
        print(f"Anomalous file: {f}")    
print(f"Il y a :\n {count_newspapers['JB838']} exemplaires du journal Le Soir,\n {count_newspapers['JB427']} exemplaires de La Libre Belgique,\n {count_newspapers['JB421']} exemplaire de L'Avenir du Luxembourg,\n {count_newspapers['JB555']} exemplaires de L'Indépendance belge,\n {count_newspapers['JB555A']} exemplaires de L'Indépendance belge (édité en Angleterre),\n {count_newspapers['JB567']} exemplaires du Journal de Bruxelles,\n {count_newspapers['JB572']} exemplaires du Journal de Charleroi,\n {count_newspapers['JB638']} exemplaires de La Meuse,\n {count_newspapers['JB685']} exemplaires du Petit Bleu,\n {count_newspapers['JB729']} exemplaire du journal Le Vingtième Siècle,\n {count_newspapers['JB773']} exemplaires de Vers l'Avenir,\n {count_newspapers['JB837']} exemplaire du journal Le Peuple,\n {count_newspapers['JB1051']} exemplaires du Drapeau Rouge")

#### Vérifier si toutes les années sont couvertes

In [ ]:
missing_years = [y for y in all_years if y not in covered_years]
print(f"Années manquantes: {', '.join(missing_years)}") #Aucune année ne manquen !

#### Voir la répartition des articles par années

In [ ]:
for year, count in count_years.items():
    print(f"Year {year}: {count} files")


#### Visualisation du nombre d'article par années

In [ ]:
# S'assurer que count_years est trié par année
sorted_years = sorted(count_years.items())

# Séparer les années et leurs comptabilisation
years, counts = zip(*sorted_years)

# Créer un index pour l'axe des y
index = np.arange(len(years))

# Tracez le graphique à barres horizontales
plt.barh(index, counts)

plt.ylabel('Années')
plt.xlabel('# documents')
plt.yticks(index, years, fontsize=8)
plt.title('Nombre de documents par année')

# Affichez le graphique
plt.show()



## Fréquence des mots dans le corpus

#### Création d'une liste de stop-words

In [ ]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout", 
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "rue", "très", "Bruxelles", "leur", "heures",
       "ans", "francs", "jours", "quelque", "toute", "jour", "bruxelles", "quelques", "grand",
       "peu", "Leurs", "mois", "temps", "ceux", "leurs", "leur", "Leurs", "lieu", "général", "grande"]
sw = set(sw)
print(f"{len(sw)} stopwords:\n {sorted(sw)}")

### Tokenization

In [ ]:
# Récupération du contenu du fichier
path = "data/all.txt"
limit = 10**8

with open(path, 'r', encoding='utf-8') as f:
    text = f.read()[:limit]

In [ ]:
# Tokenization
words = nltk.wordpunct_tokenize(text)
print(f"{len(words)} words found")

In [ ]:
words[:50]

### Calcul de la taille du vocabulaire

In [ ]:
# Eliminer les stopwords et les termes non alphabétiques
kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
voc = set(kept)
print(f"{len(kept)} words kept ({len(voc)} different word forms)")

### Récupération des mots les plus fréquents et visualisation

In [ ]:
fdist = nltk.FreqDist(kept)
fdist.most_common(10)

In [ ]:
# Plot: les n mots les plus fréquents
n = 10
fdist.plot(n, cumulative=True)

### Détection des hapax

In [ ]:
fdist.hapaxes()[:30]

In [ ]:
#Recherche des mots les plus longs du corpus
n = 30
sorted(voc, key=len, reverse=True)[:n]